## $$ Анализ~ согласованности~ данных $$

Для производственных систем часто наблюдалось, что данные вывода, используемые в производственном приложении, могут иметь некоторые различия с данными, используемыми во время обучения и проверки. Это явление известно как дрейф данных, и оно относится к неожиданному изменению структуры данных или статистических свойств набора данных, что приводит к повреждению данных и затрудняет функционирование системы ML.

Иногда может наблюдаться отклонение во всем наборе данных или в одном или нескольких функциях. Если в одном объекте есть смещение, это называется смещением объектов. Существует несколько способов обнаружения дрейфа признаков, таких как индекс стабильности популяции (PSI) (https://www.lexjansen.com/wuss/2017/47_Final_Paper_PDF.pdf ), дивергенция Кульбака–Лейблера (KL Дивергенция) (https://www.countbayesie.com/blog/2017/5/9/kullback-leibler-divergence-explained ), и метрика Вассерштейна (расстояние между землеройными движителями) (http://infolab.stanford.edu/pub/cstr/reports/cs/tr/99/1620/CS-TR-99-1620.ch4 .pdf).

In [1]:
!pip install --upgrade pandas numpy matplotlib seaborn scikit-learn deepchecks

ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\1\\anaconda3\\Lib\\site-packages\\~andas\\_libs\\algos.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Using cached pandas-1.4.3-cp39-cp39-win_amd64.whl (10.6 MB)
  Using cached numpy-1.23.2-cp39-cp39-win_amd64.whl (14.7 MB)
  Using cached scikit_learn-1.1.2-cp39-cp39-win_amd64.whl (7.4 MB)
  Created wheel for PyNomaly: filename=PyNomaly-0.3.3-py3-none-any.whl size=8498 sha256=407d53d2350a663b93b5d127d45d135729b3e3b6d221e07a296b634d8224ba16
  Stored in directory: c:\users\1\appdata\local\pip\cache\wheels\72\c2\79\a077f8b4ef4549086016b0202e61193518262377b6d23318b1
Successfully built PyNomaly
  Attempting uninstall: ipython
    Found existing installation: ipython 8.2.0
    Uninstalling ipython-8.2.0:
      Successfully uninstalled ipython-8.2.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.2
    Uninstalling pandas-1.4.2:
      Successfully uninstalled pandas-1.4.2


In [3]:
pip install Dataset

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install deepchecks


  Using cached deepchecks-0.8.2-py3-none-any.whl (3.5 MB)
  Using cached category_encoders-2.5.0-py2.py3-none-any.whl (69 kB)
  Using cached PyNomaly-0.3.3-py3-none-any.whl
  Using cached plotly-5.5.0-py2.py3-none-any.whl (26.5 MB)
  Using cached jsonpickle-2.2.0-py2.py3-none-any.whl (39 kB)
  Using cached python_utils-3.3.3-py2.py3-none-any.whl (23 kB)
  Attempting uninstall: plotly
    Found existing installation: plotly 5.6.0
    Uninstalling plotly-5.6.0:
      Successfully uninstalled plotly-5.6.0


In [6]:
from deepchecks import Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from deepchecks.suites import single_dataset_integrity, train_test_validation, model_evaluation
from deepchecks.checks import TrustScoreComparison
import scipy.stats as stats
from IPython.display import display

C:\Users\1\anaconda3\lib\site-packages\deepchecks\__init__.py:127: DeprecationWarning:

Ability to import base tabular functionality from the `deepchecks` package directly is deprecated, please import from `deepchecks.tabular` instead

C:\Users\1\anaconda3\lib\site-packages\deepchecks\suites.py:21: DeprecationWarning:

Ability to import tabular suites from the `deepchecks.suites` is deprecated, please import from `deepchecks.tabular.suites` instead

C:\Users\1\anaconda3\lib\site-packages\deepchecks\checks.py:21: DeprecationWarning:

Ability to import tabular checks from the `deepchecks.checks` is deprecated, please import from `deepchecks.tabular.checks` instead



ImportError: cannot import name 'TrustScoreComparison' from 'deepchecks.checks' (C:\Users\1\anaconda3\lib\site-packages\deepchecks\checks.py)

### О данных

Набор данных о раке молочной железы Висконсин (Диагностика) - Репозиторий машинного обучения UCI

Этот набор данных также известен как набор данных о раке молочной железы, который используется для прогнозирования наличия рака молочной железы. Это многомерный набор данных, используемый для задач, основанных на классификации, содержащий 30 различных признаков.

In [7]:
data  = datasets.load_breast_cancer(as_frame=True).frame
data.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [8]:
data.shape

(569, 31)

In [9]:
data.columns

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension',
       'target'],
      dtype='object')

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

### Изучение Подходов К Объяснению, Ориентированных На Данные

Существует множество способов реализации концепций, изученных в этой главе. Но я чувствовал, что платформа Python с открытым исходным кодом Deepchecks - это фантастическая библиотека для реализации большинства концепций, связанных с анализом согласованности и чистоты данных. Готовые методы API из фреймворка позволяют нам выполнять эти важные шаги с минимальными строками кода. В этом блокноте мы будем использовать фреймворк Deepchecks для набора данных о раке молочной железы.

In [11]:
# Let's prepare the datasets and the models
label = 'target'
train_df, test_df = train_test_split(data, test_size=0.2, random_state=123) # Performing an 80-20 split

#Creating Deepchecks object
train = Dataset(train_df, label=label)
test = Dataset(test_df, label=label)

# training and testing dataframes
x_train = train_df.drop(label, axis = 1)
y_train = train_df[label]
x_test = test_df.drop(label, axis = 1)
y_test = test_df[label]

# Model Training
model = RandomForestClassifier()
model.fit(x_train, y_train)

deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 0 categorical features were inferred.
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 0 categorical features were inferred.


RandomForestClassifier()

In [12]:
# Model Evaluation
model.score(x_test, y_test)

0.9912280701754386

К счастью, у нас есть очень хорошая модель с точностью 99% по невидимым данным. Мы можем ожидать очень ограниченных проблем, связанных с чистотой и последовательностью данных, но все же давайте проверим их с помощью платформы Deepchecks framework.

### Проверка чистоты данных с помощью пакета целостности единого набора данных Deepcheck

In [13]:
# On the training set
purity_check = single_dataset_integrity()
purity_check.run(train_dataset = train)

C:\Users\1\anaconda3\lib\site-packages\deepchecks\tabular\suites\default_suites.py:47: DeprecationWarning:

the single_dataset_integrity suite is deprecated, use the data_integrity suite instead



Accordion(children=(VBox(children=(HTML(value='\n            <h1 id="summary_Y53MW6RKDECH9JF4K01SUX0XF">Data I…

Single Dataset Integrity Suite

Набор состоит из различных проверок, таких как: Дубликаты данных, Специальные символы, Неоднозначность меток и т.д...
Каждая проверка может содержать условия (которые приведут к пропуску / сбою / предупреждению, представленному ✓ / ✖ / ! ), а также другие выходные данные, такие как графики или таблицы.
Наборы, проверки и условия могут быть изменены (см. Руководство по созданию пользовательского набора).

In [ ]:
# On the testing set
purity_check.run(test_dataset = test)

Набор состоит из различных проверок, таких как: Дубликаты данных, Специальные символы, Неоднозначность меток и т.д...
Каждая проверка может содержать условия (которые приведут к пропуску / сбою / предупреждению, представленному ✓ / ✖ / ! ), а также другие выходные данные, такие как графики или таблицы.
Наборы, проверки и условия могут быть изменены (см. Руководство по созданию пользовательского набора).

Как мы можем видеть из предыдущего шага, используя всего одну строку, мы можем выполнить тщательный анализ набора данных, чтобы обнаружить наличие пропущенных значений, дубликатов, неоднозначности меток или любых других распространенных проблем с целостностью данных. В случае, если ваша проблема требует частого использования любого другого masure для оценки целостности данных, я настоятельно рекомендую вам обратиться к владельцам фреймворка с запросом функции или даже внести свой вклад, создав pull-request и разработав этот унифицированный фреймворк.

### Проверка согласованности данных с помощью Deepchecks

Теперь мы будем использовать пакет проверки обучающих тестов для обнаружения Дрейфа данных, любых проблем с распределением данных, Наличия Утечки данных или других проблем с согласованностью данных между обучением и данными вывода с минимальными строками кода с использованием глубоких проверок.

In [15]:
data_consistency_check = train_test_validation()
data_consistency_check.run(model=model, train_dataset=train, test_dataset=test)

Accordion(children=(VBox(children=(HTML(value='\n            <h1 id="summary_SM1OEWH2NQPE6826FK2NWTXDS">Train …

Trust Score Distribution

In [21]:
pip install Checks


  Created wheel for Checks: filename=Checks-0.2-py3-none-any.whl size=3814 sha256=7facae119b41a616b6400ecb000a4672b24fa4099db20b6e10634d480d71dcc1
  Stored in directory: c:\users\1\appdata\local\pip\cache\wheels\f1\6a\a1\db1e7859ef2ef19fae601b23e02f217f22b73066d379a1f29f
Successfully built Checks


In [24]:
from Checks import TrustScoreComparison

ImportError: cannot import name 'TrustScoreComparison' from 'Checks' (C:\Users\1\anaconda3\lib\site-packages\Checks.py)

In [20]:
trust_score_distribution = TrustScoreComparison(min_test_samples = 100)
trust_score_distribution.run(train, test, model)

NameError: name 'TrustScoreComparison' is not defined

### Data Profiling

Теперь позвольте мне показать небольшую демонстрацию для выполнения простого профилирования данных. Несмотря на то, что набор данных содержит несколько функций, мы выберем 3 лучших функции на основе важности функций и создадим профили данных обучающего и тестового наборов и сравним оба профиля, чтобы выявить наличие каких-либо несоответствий.

In [25]:
# From the Data Forecastability section, the top three features are as follows:
important_features = ['worst radius', 'mean concave points', 'worst concave points']
data_profiling_train_df = train_df[important_features]
data_profiling_test_df = test_df[important_features]

In [26]:
data_profiling_train_df.head()

,worst radius,mean concave points,worst concave points
190,15.74,0.066180,0.17720
134,22.52,0.068470,0.13790
386,13.13,0.025340,0.09140
118,20.19,0.094790,0.20340
316,12.85,0.005051,0.01852


In [27]:
data_profiling_test_df.head()

,worst radius,mean concave points,worst concave points
333,12.76,0.002941,0.016670
273,10.75,0.014070,0.051590
201,20.42,0.074880,0.193900
178,14.00,0.001852,0.009259
85,22.93,0.087950,0.164200


Далее мы построим очень простой профиль данных, используя общие статистические показатели, такие как среднее значение, медиана и коэффициент вариации. Выбор сложности статистических показателей может варьироваться от набора данных к набору данных и от варианта использования к варианту использования.

In [28]:
def build_data_profile(df):
    '''
    Method to build statistical data profiles
    '''
    profile_parameter = []
    profile_value = []
    for feature in df.columns:
        # Mean
        profile_parameter.append('mean_'+ feature)
        profile_value.append(np.mean(df[feature]))
        # Median
        profile_parameter.append('median_'+ feature)
        profile_value.append(np.median(df[feature]))
        # Coefficient of Variance
        profile_parameter.append('cov_'+ feature)
        profile_value.append(np.std(df[feature]/np.mean(df[feature])))
     
    data_profile_df = pd.DataFrame([profile_value], columns = profile_parameter)
    return data_profile_df

In [29]:
train_profile = build_data_profile(data_profiling_train_df)
train_profile

,mean_worst radius,median_worst radius,cov_worst radius,mean_mean concave points,median_mean concave points,cov_mean concave points,mean_worst concave points,median_worst concave points,cov_worst concave points
0,16.349314,15.05,0.297475,0.050018,0.034,0.777637,0.116275,0.1001,0.567213


In [30]:
test_profile = build_data_profile(data_profiling_test_df)
test_profile

,mean_worst radius,median_worst radius,cov_worst radius,mean_mean concave points,median_mean concave points,cov_mean concave points,mean_worst concave points,median_worst concave points,cov_worst concave points
0,15.949395,14.34,0.293118,0.044534,0.032035,0.851979,0.107946,0.09833,0.594014


In [31]:
# Calculate absolute percentage change in the DataFrame column values
pct_df = pd.DataFrame(columns = ['Parameter', '% change'])
for col in train_profile.columns:
    pct_dict = {'Parameter' : col, '% change' : np.abs((test_profile[col][0] - train_profile[col][0])/train_profile[col][0] * 100)}
    pct_df = pct_df.append(pct_dict, ignore_index = True)

display(pct_df)     

C:\Users\1\AppData\Local\Temp\ipykernel_4604\1924948186.py:5: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\1\AppData\Local\Temp\ipykernel_4604\1924948186.py:5: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\1\AppData\Local\Temp\ipykernel_4604\1924948186.py:5: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\1\AppData\Local\Temp\ipykernel_4604\1924948186.py:5: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\1\AppData\Local\Temp\ipykernel_4604\1924948186.py:5: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\1\AppD

,Parameter,% change
0,mean_worst radius,2.446094
1,median_worst radius,4.717608
2,cov_worst radius,1.464589
3,mean_mean concave points,10.964007
4,median_mean concave points,5.779412
5,cov_mean concave points,9.559889
6,mean_worst concave points,7.162983
7,median_worst concave points,1.768232
8,cov_worst concave points,4.724976


Из приведенной выше таблицы мы можем ясно видеть, что для всех статистических показателей абсолютное процентное изменение значений профиля между поездом и тестовым набором составляет менее 25%. Если абсолютное процентное изменение составляет более 20%, это указывает на наличие некоторого смещения данных в наборе данных.

Анализ чистоты данных

Ориентированный на данные XAI также включает в себя другие параметры, которые могут быть проанализированы, такие как состязательная надежность (https://adversarial-ml-tutorial.org/introduction /), сравнение показателей доверия (https://arxiv.org/abs/1805.11783 ), ковариационный сдвиг (https://arxiv.org/abs/2111.08234 ), утечка данных между обучающими и проверочными наборами данных (https://machinelearningmastery.com/data-leakage-machine-learning /), и анализ чувствительности производительности модели на основе изменений данных. Все эти концепции применимы как к табличным данным, так и к неструктурированным данным, таким как изображения и текст.

### Тщательный анализ данных и процесс профилирования

В соответствии с принципами подходов, ориентированных на объяснение данных, сначала мы пытаемся выполнить тщательный анализ базового набора данных. Мы пытаемся случайным образом перетасовывать данные для создания различных обучающих и проверочных наборов и наблюдаем за любыми эффектами переобучения или недообучения. Если модель переоснащена или недостаточно приспособлена, очевидно, что для обобщения модели требуется больше данных. Если имеющихся данных недостаточно по объему, существуют способы генерирования синтетических или искусственных данных. Одним из таких популярных методов, используемых для классификации изображений, является увеличение данных (https://research .aimultiple.com/data-augmentation /). Метод синтетической передискретизации меньшинства (SMOTE) (https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification /) также является мощным методом, который вы можете использовать для увеличения размера набора данных. Некоторые из этих подходов, ориентированных на данные, обычно применяются во время обычных рабочих процессов ML. Однако нам необходимо осознать важность этих шагов для объяснимости моделей черного ящика.

Иногда выбор статистических показателей может быть затруднен. Итак, мы обычно используем некоторые популярные показатели распределения, чтобы обнаружить наличие дрейфа данных, используя количественный подход. Один из таких показателей называется распределением баллов доверия (https://arxiv.org/abs/1805.11783 ). Есть в Deepcheks Trust Score D.

В идеале распределения должны быть почти одинаковыми как для обучающих, так и для тестовых наборов данных. Однако, если распределение баллов доверия для набора выводов смещено в крайнюю левую сторону, это указывает на то, что обученная модель имеет меньшую уверенность в данных вывода, тем самым намекая на наличие дрейфа. Если распределение показателей доверия к данным вывода смещено в крайнюю правую сторону, возможно, с моделью возникла какая-то проблема и существует высокая вероятность утечки данных, поскольку в идеале обученная модель не может быть более уверенной в тестовых данных по сравнению с обучающими данными.

Для обнаружения смещения признаков в категориальных признаках популярным выбором показателя является индекс стабильности населения (PSI) (https://www.lexjansen.com/wuss/2017/47_Final_Paper_PDF.pdf ). Это статистический метод, используемый для измерения изменения переменной за определенный период времени. Если общий показатель дрейфа превышает 0,2 или 20%, то дрейф считается значительным, что свидетельствует о наличии дрейфа признаков.

Для обнаружения смещений признаков в числовых признаках используется метрика Вассерштейна (https://kowshikchilamkurthy.medium.com/wasserstein-distance-contraction-mapping-and-modern-rl-theory-93ef740ae867 ) является популярным выбором. Это функция расстояния для измерения расстояния между двумя распределениями вероятностей. Аналогично PSI, если показатель дрейфа с использованием метрики Вассерштейна превышает 20%, это считается значительным, и числовой признак считается имеющим признак дрейфа.

Аналогичные концептуальные отклонения также могут быть обнаружены с использованием этих показателей. Для задач регрессии эффективна метрика Вассерштейна, в то время как для задач классификации более эффективен PSI.

In [34]:
!pip install --upgrade pandas numpy matplotlib seaborn numba==0.53 adversarial-robustness-toolbox tensorflow sewar

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\1\anaconda3\python.exe' 'C:\Users\1\AppData\Local\Temp\pip-standalone-pip-410vsgbn\__env_pip__.zip\pip' install --ignore-installed --no-user --prefix 'C:\Users\1\AppData\Local\Temp\pip-build-env-z7od4ucy\overlay' --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- setuptools wheel 'Cython>=0.28.5' 'numpy==1.13.3; python_version=='"'"'3.6'"'"' and platform_system!='"'"'AIX'"'"' and platform_python_implementation == '"'"'CPython'"'"'' 'numpy==1.14.0; python_version=='"'"'3.6'"'"' and platform_system!='"'"'AIX'"'"' and platform_python_implementation != '"'"'CPython'"'"'' 'numpy==1.14.5; python_version=='"'"'3.7'"'"' and platform_system!='"'"'AIX'"'"'' 'numpy==1.17.3; python_version>='"'"'3.8'"'"' and platform_system!='"'"'AIX'"'"'' 'numpy==1.16.0; python_version=='"'"'3.6'"'"' and platform_system=='"'"'AIX'"'"'' 'numpy==1.16.0; python_version=='"'"'3.7'"'"' and platform_syste

  Using cached numpy-1.23.2-cp39-cp39-win_amd64.whl (14.7 MB)
  Using cached matplotlib-3.5.3-cp39-cp39-win_amd64.whl (7.2 MB)

  Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX" and platform_python_implementation == "CPython"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX" and platform_python_implementation != "CPython"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.7" and platform_system != "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.6" and platform_system == "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.7" and platform_system == "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version >= "3.8" and platform_system == "AIX"' don't match your environment
    Using cached wheel-0.37.1-py2.py3-none-any.whl (35 kB)
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\1\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\1\\AppD


  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'


  
    C:\Users\1\AppData\Local\Temp\pip-install-bive7hk3\numpy_940347872e2b4275aa09de1b9904a7b7\numpy\distutils\system_info.py:690: UserWarning:
        Blas (http://www.netlib.org/blas/) sources not found.
        Directories to search for the sources can be specified in the
        numpy/distutils/site.cfg file (section [blas_src]) or by setting
        the BLAS_SRC environment variable.
      self.calc_info()
      NOT AVAILABLE
  
    "svnversion" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
    Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
    non-existing path in 'numpy\\distutils': 'site.cfg'
    lapack_opt_info:
    lapack_mkl_info:
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
    customize MSVCCompiler
      libraries mkl_rt not found in ['C:\\Users\\1\\anaconda3\\lib', 'C:\\', 'C:\\Users\\1\\anaconda3\\libs']
      NOT AVAILABLE
  
    openblas_lapack_info:
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; 

  Created wheel for sewar: filename=sewar-0.4.5-py3-none-any.whl size=10602 sha256=600f9d46b4e8435a384a1c007ce779cce76133dc54eb1c46755c4478b428fb20
  Stored in directory: c:\users\1\appdata\local\pip\cache\wheels\90\c1\d2\90faa299f5473d447a9a489377305840b7268c92b79bb5e000
Successfully built sewar
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.38.0



    customize MSVCCompiler
      libraries lapack_atlas not found in C:\Users\1\anaconda3\libs
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
    customize MSVCCompiler
      libraries satlas,satlas not found in C:\Users\1\anaconda3\libs
    <class 'numpy.distutils.system_info.atlas_3_10_info'>
      NOT AVAILABLE
  
    atlas_threads_info:
    Setting PTATLAS=ATLAS
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
    customize MSVCCompiler
      libraries lapack_atlas not found in C:\Users\1\anaconda3\lib
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
    customize MSVCCompiler
      libraries ptf77blas,ptcblas,atlas not found in C:\Users\1\anaconda3\lib
    No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
    customize MSVCCompiler
      libraries lapack_atlas not found in C:\
    No module named 'numpy.di

In [36]:
pip install art

In [38]:
pip install art.estimators

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement art.estimators (from versions: none)
ERROR: No matching distribution found for art.estimators


In [37]:
import warnings
warnings.filterwarnings('ignore')

from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, CarliniLInfMethod
from art.defences.preprocessor import SpatialSmoothing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sewar.full_ref import psnr, uqi, ergas
from IPython.display import Image, display

from tensorflow import keras
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import Xception, preprocess_input, decode_predictions

ModuleNotFoundError: No module named 'art.estimators'

In [ ]:
if tf.__version__[0] != '2':
    raise ImportError('TensorFlow v2.0+ is required by the notebook.')

print('Eager execution enabled: ', tf.executing_eagerly()) 

В этом примере мы собираемся использовать предварительно обученную модель ImageNet из TensorFlow, поскольку мы моделируем ситуацию, когда обученная модель и данные вывода подвергаются атакам противника. Мы собираемся использовать Python-фреймворк adversarial-robustness-toolbox для создания примеров adversarial и использовать sewar - фреймворк Python для оценки качества изображения. Изображение, подвергнутое атаке противника, будет иметь ухудшение качества или отклонение в статистических свойствах исходного изображения, и, следовательно, мы собираемся обнаружить пример противника с помощью этой структуры.

In [ ]:
def load_image_data_from_url(url):
    '''
    Function to load image data from online
    '''
    # The local path to our target image
    image_path = keras.utils.get_file(
    "inference_data.jpg", url
    )

    display(Image(image_path))
    return image_path

image_path = load_image_data_from_url(url = 'https://i.imgur.com/JT7SP0M.jpeg')

In [ ]:
IMG_SIZE = (299, 299)
def transform_image(image_path, size):
    '''
    Function to transform an image to normalized numpy array
    '''
    img = image.load_img(image_path, target_size=size)
    img = image.img_to_array(img)# Transforming the image to get the shape as [channel, height, width]
    img = np.expand_dims(img, axis=0) # Adding dimension to convert array into a batch of size (1,299,299,3)
    img = img/255.0 # normalizing the image to keep within the range of 0.0 to 1.0
    
    return img

normalized_img = transform_image(image_path, IMG_SIZE)

Определение модели

В этом примере мы не обучаем модель с нуля, а скорее определяем предварительно обученную модель Tensorflow Xception, чтобы продемонстрировать влияние враждебных атак на данные вывода.

In [ ]:
model = Xception(weights="imagenet")

def get_model_predictions(data):
    model_prediction = model.predict(data)
    print(f"The predicted class is : {decode_predictions(model_prediction, top=1)[0][0][1]}")
    return decode_predictions(model_prediction, top=1)[0][0][1]
    
pred_orig = get_model_predictions(normalized_img)

Генерация примеров состязательности с использованием инструментария Состязательной надежности

In [ ]:
art_model = KerasClassifier(model=model, clip_values=(0, 1))

Using FGSM

In [ ]:
attack_fgsm = FastGradientMethod(art_model, eps = 0.1)
adv_images_fgsm = attack_fgsm.generate(x = normalized_img)
pred_fgsm = get_model_predictions(adv_images_fgsm)

Using C&W method

In [ ]:
attack_cw = CarliniLInfMethod(art_model, max_iter=100, verbose=True)
adv_images_cw = attack_cw.generate(x = normalized_img, verbose=True)
pred_cw =get_model_predictions(adv_images_cw)

Using PGD

In [ ]:
norm = 2
attack_pgd = ProjectedGradientDescent(art_model, eps = 0.1, verbose=True)
adv_images_pgd = attack_pgd.generate(x = normalized_img, verbose=True)
pred_pgd = get_model_predictions(adv_images_pgd)

Displaying the adversarial examples

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(18,12))
ax1.imshow(normalized_img[0])
ax1.set_title(f'Original. Predicted class: {pred_orig}')
ax1.set_axis_off()
ax2.imshow(adv_images_fgsm[0])
ax2.set_title(f'FGSM Sample. Predicted class: {pred_fgsm}')
ax2.set_axis_off()
ax3.imshow(adv_images_cw[0])
ax3.set_title(f'C&W Sample. Predicted class: {pred_cw}')
ax3.set_axis_off()
ax4.imshow(adv_images_pgd[0])
ax4.set_title(f'PGD Sample. Predicted class: {pred_pgd}')
ax4.set_axis_off()
plt.tight_layout()

Сравнение качества изображения состязательных примеров с исходным изображением
Для сравнения качества изображения исходного образца с примерами состязательности мы будем использовать пакет Sewar: https://pypi.org/project/sewar / ad использует показатели PSNR, UQI и ERGAS для оценки разницы в качестве изображения между состязательной выборкой и фактическими выборками. Более подробную информацию об этих и других показателях можно найти по адресу: https://sewar.readthedocs.io/en/latest /.

In [ ]:
# Calculate the image quality scores
image_quality_df = pd.DataFrame(columns = ['Sample', 'PSNR', 'ERGAS', 'UQI'])
# Original Sample
image_quality_dict = {'Sample' : 'Original', 
                     'PSNR' : psnr((normalized_img*255).astype('int'), (normalized_img*255).astype('int')),
                     'ERGAS' : ergas((normalized_img[0]*255).astype('int'), (normalized_img[0]*255).astype('int')),
                     'UQI' : uqi(normalized_img[0], normalized_img[0])
                     }
image_quality_df = image_quality_df.append(image_quality_dict, ignore_index = True)
# FGSM Sample
image_quality_dict = {'Sample' : 'FGSM', 
                     'PSNR' : psnr((normalized_img*255).astype('int'), (adv_images_fgsm*255).astype('int')),
                     'ERGAS' : ergas((normalized_img[0]*255).astype('int'), (adv_images_fgsm[0]*255).astype('int')),
                     'UQI' : uqi(normalized_img[0], adv_images_fgsm[0])
                     }
image_quality_df = image_quality_df.append(image_quality_dict, ignore_index = True)
# C&W Sample
image_quality_dict = {'Sample' : 'C&W', 
                     'PSNR' : psnr((normalized_img*255).astype('int'), (adv_images_cw*255).astype('int')),
                     'ERGAS' : ergas((normalized_img[0]*255).astype('int'), (adv_images_cw[0]*255).astype('int')),
                     'UQI' : uqi(normalized_img[0], adv_images_cw[0])
                     }
image_quality_df = image_quality_df.append(image_quality_dict, ignore_index = True)
# PGD
image_quality_dict = {'Sample' : 'PGD', 
                     'PSNR' : psnr((normalized_img*255).astype('int'), (adv_images_pgd*255).astype('int')),
                     'ERGAS' : ergas((normalized_img[0]*255).astype('int'), (adv_images_pgd[0]*255).astype('int')),
                     'UQI' : uqi(normalized_img[0], adv_images_pgd[0])
                     }
image_quality_df = image_quality_df.append(image_quality_dict, ignore_index = True)
display(image_quality_df)    

Для этой демонстрации мы использовали состязательные атаки с использованием методов FGSM, C&W и PGD. Другие формы состязательных атак, предоставляемые инструментарием состязательной надежности, можно просмотреть в официальной документации. Мы можем заметить, что из-за враждебной атаки первоначальное модельное предсказание индийского слона было изменено на Бивень и трицератопс, которые неверны. Бивень все еще кажется в порядке, хотя на изображении у слона нет никакого бивня! Но предсказание Мейза было далеко не так близко.

Когда мы пытаемся провести визуальное качественное сравнение, мы видим, что сгенерированная FGSM выборка имеет наибольшее количество возмущений, за которыми следуют PGD и C& W. Образцы, сгенерированные C & W, выглядят более реалистично, и даже экспертам-людям трудно обнаружить состязательную выборку для образцов, сгенерированных C & W.

Теперь, когда мы сравниваем метрические оценки сгенерированных состязательных выборок с исходными данными, мы видим, что метрика ERGAS дает нам четкое представление о количестве возмущений.

Предложения по повышению надежности состязательности
Как обсуждалось в главе 03 в разделе Методы повышения состязательной надежности, наиболее распространенными подходами к повышению состязательной надежности моделей являются следующие способы:

Пространственное сглаживание
Состязательное обучение
Передача знаний с помощью состязательных примеров
В целях демонстрации в этом блокноте мы увидим только использование Пространственного сглаживания для повышения надежности модели в условиях состязательности.

In [ ]:
# Defending using Spatial Smoothing of the samples
spatial_smoothing = SpatialSmoothing(window_size=9)
# FGSM
smooth_fgsm, _ = spatial_smoothing(adv_images_fgsm)
pred_fgsm_ss = get_model_predictions(smooth_fgsm)
# C&W
smooth_cw, _ = spatial_smoothing(adv_images_cw)
pred_cw_ss = get_model_predictions(smooth_cw)
# PGD
smooth_pgd, _ = spatial_smoothing(adv_images_pgd)
pred_pgd_ss = get_model_predictions(smooth_pgd)

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(18,12))

#fig.suptitle('Adversarial samples with the original')
ax1.imshow(normalized_img[0])
ax1.set_title(f'Original. Predicted: {pred_orig}')
ax1.set_axis_off()
ax2.imshow(smooth_fgsm[0])
ax2.set_title(f'Smoothened FGSM Sample. Predicted : {pred_fgsm_ss}')
ax2.set_axis_off()
ax3.imshow(smooth_cw[0])
ax3.set_title(f'Smoothened C&W Sample. Predicted : {pred_cw_ss}')
ax3.set_axis_off()
ax4.imshow(smooth_pgd[0])
ax4.set_title(f'Smoothened PGD Sample. Predicted : {pred_pgd_ss}')
ax4.set_axis_off()
plt.tight_layout()

In [ ]:
# Calculate the image quality scores
image_quality_df = pd.DataFrame(columns = ['Sample', 'PSNR', 'ERGAS', 'UQI'])
# Original Sample
image_quality_dict = {'Sample' : 'Original', 
                     'PSNR' : psnr((normalized_img*255).astype('int'), (normalized_img*255).astype('int')),
                     'ERGAS' : ergas((normalized_img[0]*255).astype('int'), (normalized_img[0]*255).astype('int')),
                     'UQI' : uqi(normalized_img[0], normalized_img[0])
                     }
image_quality_df = image_quality_df.append(image_quality_dict, ignore_index = True)
# FGSM Sample
image_quality_dict = {'Sample' : 'FGSM', 
                     'PSNR' : psnr((normalized_img*255).astype('int'), (smooth_fgsm*255).astype('int')),
                     'ERGAS' : ergas((normalized_img[0]*255).astype('int'), (smooth_fgsm[0]*255).astype('int')),
                     'UQI' : uqi(normalized_img[0], smooth_fgsm[0])
                     }
image_quality_df = image_quality_df.append(image_quality_dict, ignore_index = True)
# C&W Sample
image_quality_dict = {'Sample' : 'C&W', 
                     'PSNR' : psnr((normalized_img*255).astype('int'), (smooth_cw*255).astype('int')),
                     'ERGAS' : ergas((normalized_img[0]*255).astype('int'), (smooth_cw[0]*255).astype('int')),
                     'UQI' : uqi(normalized_img[0], smooth_cw[0])
                     }
image_quality_df = image_quality_df.append(image_quality_dict, ignore_index = True)
# PGD
image_quality_dict = {'Sample' : 'PGD', 
                     'PSNR' : psnr((normalized_img*255).astype('int'), (smooth_pgd*255).astype('int')),
                     'ERGAS' : ergas((normalized_img[0]*255).astype('int'), (smooth_pgd[0]*255).astype('int')),
                     'UQI' : uqi(normalized_img[0], smooth_pgd[0])
                     }
image_quality_df = image_quality_df.append(image_quality_dict, ignore_index = True)
display(image_quality_df)

В этом примере, хотя пространственное сглаживание не сильно помогло, но оно изменило выборку, атакованную методом PGD, который предсказал результат как трицератопс, а вместо этого результат предсказан как Индийский слон, что правильно. Другие подходы к повышению надежности состязательности, такие как состязательное обучение, которое предоставляет состязательные выборки с использованием увеличения данных, более эффективны, но требуют много времени, поскольку обучение проводится с нуля. Аналогичным образом, передача обучения с использованием состязательных образцов также может быть эффективной для повышения состязательной надежности моделей. Но мониторинг враждебных атак является важной частью Data Centric XAI.

Заключительные мысли
В этой тетради вы узнали, как враждебные атаки могут нанести ущерб результату модели. Вы также узнали, как создавать состязательные выборки с использованием различных методов, предоставляемых инструментарием состязательной надежности, и как оценивать или сравнивать поврежденные выборки с исходными данными. Наконец, мы обсудили использование пространственного Sm
